# OpenACC Interoperability

This lab is intended for C/C++ programmers. If you prefer to use Fortran, click [this link.](../Fortran/README.ipynb)

---
## Introduction

The primary goal of this lab is to cover how to write an OpenACC code to work alongside other CUDA codes and accelerated libraries. There are several ways to make an OpenACC/CUDA interoperable code, and we will go through them one-by-one, with a short exercise for each.

When programming in OpenACC, the distinction between CPU/GPU memory is abstracted. For the most part, you do not need to worry about explicitly differentiating between CPU and GPU pointers; the OpenACC runtime handles this for you. However, in CUDA, you do need to differentiate between these two types of pointers. Let's start with using CUDA allocated GPU data in our OpenACC code.

---

## OpenACC Deviceptr Clause

The OpenACC `deviceptr` clause is used with the `data`, `parallel`, or `kernels` directives. It can be used in the same way as other data clauses such as `copyin`, `copyout`, `copy`, or `present`. The `deviceptr` clause is used to specify that a pointer is not a host pointer but rather a device pointer.

This clause is important when working with OpenACC + CUDA interoperability because it is one way we can operate on CUDA allocated device data within an OpenACC code. Take the following example:

**Allocation with CUDA**

```c++
double *cuda_allocate(int size) {
    double *ptr;
    cudaMalloc((void**) &ptr, size * sizeof(double));
    return ptr;
}
```

**Parallel Loop with OpenACC**

```c++
int main() {
    double *cuda_ptr = cuda_allocate(100); // Allocated on the device, but not the host!
    
    #pragma acc parallel loop deviceptr(cuda_ptr)
    for(int i = 0; i < 100; i++) {
        cuda_ptr[i] = 0.0;
    }
}
```

Normally, the OpenACC runtime expects to be given a host pointer, which will then be translated to some associated device pointer. However, when using CUDA to do our data management, we do not have that connection between host and device. The `deviceptr` clause is a way to tell the OpenACC runtime that a given pointer should not be translated since it is already a device pointer.

To practice using the `deviceptr` clause, we have a short exercise. We will examine two functions, both compute a dot product. The first code is [dot.c](/edit/C/deviceptr/dot.c), which is a serial dot product. Next is [dot_acc.c](/edit/C/deviceptr/dot_acc.c), which is an OpenACC parallelized version of dot. Both dot and dot_acc are called from [main.cu](/edit/C/deviceptr/main.cu) (*note: .cu is the conventional extension for a CUDA C++ source file*). In main.cu, we use host pointers to call dot, and device pointers to call dot_acc. Let's quickly run the code, it will produce an error.

In [ ]:
!make -C deviceptr

To fix the error, we must tell the OpenACC runtime in the dot_acc function that our pointers are device pointers. Edit the [dot_acc.c](/edit/C/deviceptr/dot_acc.c) file using the deviceptr clause to get the code working. When you think you have it, run the code below and see if the error is fixed.

In [ ]:
!make -C deviceptr

Next, let's do the opposite. Let's take data that was allocated with OpenACC, and use it in a CUDA function.

---
## OpenACC host_data directive

The `host_data` directive is used to make the OpenACC mapped device address available to the host. There are a few clauses that can be used with host_data, but the one that we are interested in using is `use_device`. We will use the `host_data` directive with the `use_device` clause to grab the underlying device pointer that OpenACC usually abstracts for us. Then we can use this device pointer to pass to CUDA kernels or to use accelerated libraries. Let's look at a code example:

**Inside CUDA Code**

```c++
__global__
void example_kernel(int *A, int size) {
    // Kernel Code
}

extern "C" void example_cuda(int *A, int size) {
    example_kernel<<<512,128>>>(A, size);
}
```

**Inside OpenACC Code**

```c++
extern void example_cuda(int*, int);

int main() {
    int *A = (int*) malloc(100*sizeof(int));
    #pragma acc data create(A[0:100])
    {
        #pragma acc host_data use_device(A)
        {
            example_cuda(A, 100);
        }
    }
}
```

A brief rundown of what is actually happening under-the-hood: the `data` directive creates a device copy of the array `A`, and the host pointer of `A` is linked to the device pointer of `A`. This is typical OpenACC behavior. Next, the `host_data use_device` translates the `A` variable on the host to the device pointer so that we can pass it to our CUDA function.

To practice this, let's work on another code. We still have [dot.c](/edit/C/host_data/dot.c) for our serial code. But instead of an OpenACC version of dot, we have a CUDA version in [dot_kernel.cu](/edit/C/host_data/dot_kernel.cu). Both of these functions are called in [main.c](/edit/C/host_data/main.c). First, let's run the code and see the error.

In [ ]:
!make -C host_data

Now edit [main.c](/edit/C/host_data/main.c) and use the `host_data` and `use_device` to pass device pointers when calling our CUDA function. When you're ready, rerun the code below, and see if the error is fixed.

In [ ]:
!make -C host_data

---
## Using cuBLAS with OpenACC

We are also able to use accelerated libraries with `host_data` and `use_device` as well. Just like the previous section, we can allocate the data with OpenACC using either the `data` or `enter data` directives. Then, pass that data to a cuBLAS call with `host_data`. This code is slightly different than before; we will be working on a matrix multiplication code. The serial code is found in [matmult.c](/edit/C/cublas/matmult.c). The cuBLAS code is in [matmult_cublas.cu](/edit/C/cublas/matmult_cublas.cu). Both of these are called from [main.c](/edit/C/cublas/main.c). Let's try running the code and seeing the error.

In [ ]:
!make -C cublas

Now, edit [main.c](/edit/C/cublas/main.c) and use host_data/use_device on the cublas call (similar to what you did in the previous exercise). Rerun the code below when you're ready, and see if the error is fixed.

In [ ]:
!make -C cublas

Next we will learn how make CUDA allocated memory behave like OpenACC allocated memory.

---
## OpenACC map_data

We briefly mentioned earlier about how OpenACC creates a mapping between host and device memory. When using CUDA allocated memory within OpenACC, that mapping is not created automatically, but it can be created manually. We are able to map a host pointer to a device pointer by using the OpenACC `acc_map_data` function. Then, before the data is unallocated, you will use `acc_unmap_data` to undo the mapping. Let's look at a quick example.


**Inside CUDA Code**

```c++
int *cuda_allocate(int size) {
    int *ptr;
    cudaMalloc((void**) &ptr, size*sizeof(int));
    return ptr;
}

void cuda_deallocate(int* ptr) {
    cudaFree(ptr);
}
```

**Inside OpenACC Code**

```c++
int main() {
    int *A = (int*) malloc(100 * sizeof(int));
    int *A_device = cuda_allocate(100);
    acc_map_data(A, A_device, 100*sizeof(int));
    
    #pragma acc parallel loop present(A[0:100])
    for(int i = 0; i < 100; i++) {
        // Computation
    }
    
    acc_unmap_data(A);
    cuda_deallocate(A_device);
    free(A);
}
```

To practice, we have another example code which uses the `dot` functions again. Serial `dot` is in [dot.c](/edit/C/map/dot.c). OpenACC `dot` is in [dot_acc.c](/edit/C/map/dot_acc.c). Both of them are called from [main.cu](/edit/C/map/main.cu). Since main is a CUDA code, we have placed the OpenACC map/unmap in a separate file [map.c](/edit/C/map/map.c). Try running the code and seeing the error.

In [ ]:
!make -C map

Now, edit [map.c](/edit/C/map/map.c) and add the OpenACC mapping functions. When you're ready, rerun the code below and see if the error is fixed.

In [ ]:
!make -C map

---
## Routine

The last topic to discuss is using CUDA device functions within OpenACC `parallel` and `kernels` regions. These are functions that are compiled to be called from the accelerator within a GPU kernel or OpenACC region.

If you want to compile an OpenACC function to be used on the device, you will use the `routine` directive with the following syntax:

```c++
#pragma acc routine seq
int func() {
    return 0;
}
```

You can also have a function with a loop you want to parallelize like so:

```c++
#pragma acc routine vector
int func() {
    int sum = 0;
    #pragma acc loop vector
    for(int i = 0; i < 100; i++) {
        sum += i;
    }
    return sum;
}
```

To use CUDA `__device__` functions within our OpenACC loops, we can also use the `routine` directive. See the following example:

**In CUDA Code**

```c++
extern "C" __device__
int cuda_func(int x) {
        return x*x;
}
```
**In OpenACC Code**

```c++
#pragma acc routine seq
extern int cuda_func(int);

...

int main() {
    A = (int*) malloc(100 * sizeof(int));
    #pragma acc parallel loop copyout(A[:100])
    for(int i = 0; i < 100; i++) {
        A[i] = cuda_func(i);
    }
}
```

To practice, we have one last code to try out. Our main function is in [main.c](/edit/C/routine/main.c), and our serial code is in [distance_map.c](/edit/C/routine/distance_map.c). Our parallel loop is in [distance_map_acc.c](/edit/C/routine/distance_map_acc.c). Note that the parallel loop is trying to use a CUDA `__device__` function without including any routine information. The CUDA function is in [dist_cuda.cu](/edit/C/routine/dist_cuda.cu). Let's run the code and see the error.

In [ ]:
!make -C routine

Now, edit [distance_map_acc.c](/edit/C/routine/distance_map_acc.c) and include the routine directive. When you're ready, rerun the code below and see if the error is fixed.

In [ ]:
!make -C routine

---
## Bonus Task

Here are some additional resources for OpenACC/CUDA interoperability:

[This is an NVIDIA devblog about some common techniques for implementing OpenACC + CUDA](https://devblogs.nvidia.com/3-versatile-openacc-interoperability-techniques/)

[This is a github repo with some additional code examples demonstrating the lessons covered in this lab](https://github.com/jefflarkin/openacc-interoperability)

---

## Post-Lab Summary

If you would like to download this lab for later viewing, it is recommend you go to your browsers File menu (not the Jupyter notebook file menu) and save the complete web page.  This will ensure the images are copied down as well.

You can also execute the following cell block to create a zip-file of the files you've been working on, and download it with the link below.

In [ ]:
%%bash
rm -f openacc_files.zip
zip -r openacc_files.zip *

**After** executing the above zip command, you should be able to download the zip file [here](files/openacc_files.zip)